In [1]:
import os

In [2]:
%pwd

'/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk


/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-06-20 20:21:07,494 - INFO - PyTorch version 2.7.1 available.


In [ ]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        ## loading the data 
        dataset_samsum_pt= load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=int(float(self.config.save_steps)) ,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer= Trainer(
        model=model,
        args=trainer_args,
        tokenizer=tokenizer,
        data_collator=seq2seq_data_collator,
        train_dataset=dataset_samsum_pt["test"],
        eval_dataset=dataset_samsum_pt["validation"]
    )
        trainer.train()
        model.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [16]:
config= ConfigurationManager() 
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

2025-06-20 20:34:58,583 - INFO - yaml file: config/config.yaml loaded successfully
2025-06-20 20:34:58,587 - INFO - yaml file: params.yaml loaded successfully
2025-06-20 20:34:58,588 - INFO - created directory at: artifacts
2025-06-20 20:34:58,589 - INFO - created directory at: artifacts/model_trainer


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: invalid literal for int() with base 10: '1e6'

In [ ]:
!!pip install --upgrade transformers


['Requirement already satisfied: transformers in ./venv/lib/python3.10/site-packages (4.52.4)',
 'Requirement already satisfied: filelock in ./venv/lib/python3.10/site-packages (from transformers) (3.18.0)',
 'Requirement already satisfied: huggingface-hub<1.0,>=0.30.0 in ./venv/lib/python3.10/site-packages (from transformers) (0.33.0)',
 'Requirement already satisfied: numpy>=1.17 in ./venv/lib/python3.10/site-packages (from transformers) (2.2.6)',
 'Requirement already satisfied: packaging>=20.0 in ./venv/lib/python3.10/site-packages (from transformers) (25.0)',
 'Requirement already satisfied: pyyaml>=5.1 in ./venv/lib/python3.10/site-packages (from transformers) (6.0.2)',
 'Requirement already satisfied: regex!=2019.12.17 in ./venv/lib/python3.10/site-packages (from transformers) (2024.11.6)',
 'Requirement already satisfied: requests in ./venv/lib/python3.10/site-packages (from transformers) (2.32.4)',
 'Requirement already satisfied: tokenizers<0.22,>=0.21 in ./venv/lib/python3.1

In [ ]:
##!pip install torch torchvision torchaudio


  Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl.metadata (29 kB)
Using cached torch-2.7.1-cp310-none-macosx_11_0_arm64.whl (68.6 MB)


In [11]:
import transformers
print(transformers.__version__)


4.52.4
